In [ ]:
# Function to calculate sentiment score for a single text
def calculate_sentiment(input_text):
    text = input_text.split()
    from afinn import Afinn
    from collections import defaultdict

    # Load AFINN lexicon with defaultdict to handle missing words
    sent_lexicon = defaultdict(lambda: 0, Afinn()._dict)

    #print(*(sent_lexicon[word] for word in text), sep='\n')
    
    # Sum up the sentiment scores of each word in the text
    sentiment_score = sum(sent_lexicon[word] for word in text)
    
    # Optionally normalize by the number of words (average sentiment per word)
    if len(text) > 0:
        normalized_score = sentiment_score / len(text)
    else:
        normalized_score = 0

    # if normalized_score > 0:
    #     print('text:', text)
    #     print('sentiment_score:', sentiment_score)
    #     print('len(text):', len(text))
    #     print('normalized_score:', normalized_score)
        
    return normalized_score

De her to er også en mulighed

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def calculate_sentiment_vader(input_text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(input_text)
    return sentiment['compound']  # Returns a score between -1 (negative) and +1 (positive)

In [ ]:
from textblob import TextBlob

def calculate_sentiment_textblob(input_text):
    blob = TextBlob(input_text)
    return blob.sentiment.polarity  # Returns a score between -1 and +1